#### Building a RAG chatbot system for the constitution of Kenya

In [9]:
from pdfminer.high_level import extract_text

pdf_path = "The_Constitution_of_Kenya_2010.pdf"
text = extract_text(pdf_path)

# Save to .txt for reuse
with open("constitution_of_kenya.txt", "w", encoding="utf-8") as f:
    f.write(text)

In [10]:
print(text[:500])

LAWS OF KENYA

THE CONSTITUTION OF KENYA, 2010

Published by the National Council for Law Reporting
with the Authority of the Attorney-General

www.kenyalaw.org

Constitution of Kenya, 2010

THE CONSTITUTION OF KENYA, 2010

ARRANGEMENT OF ARTICLES

PREAMBLE

CHAPTER ONE—SOVEREIGNTY OF THE PEOPLE AND 

SUPREMACY OF THIS CONSTITUTION

1—Sovereignty of the people.
2—Supremacy of this Constitution.
3—Defence of this Constitution.

CHAPTER TWO—THE REPUBLIC

4—Declaration of the Republic.
5—Territory


In [11]:
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from pdfminer.high_level import extract_text

In [12]:
pdf_path = "The_Constitution_of_Kenya_2010.pdf"
text = extract_text(pdf_path)

# Save to .txt for reuse
with open("constitution_of_kenya.txt", "w", encoding="utf-8") as f:
    f.write(text)

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(text)

In [15]:
# Step 2:
# Embed and Store in ChromaDB ===
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_texts(chunks, embedding_model)

In [ ]:
# Step 3:
# Retrieve Relevant Chunks ===
def retrieve_context(query, k=4):
    results = db.similarity_search(query, k=k)
    return "\n".join([doc.page_content for doc in results])

In [17]:
#  step 4:
#  Query Groq API 
def query_groq(context, question, api_key):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    prompt = f"""Use the following excerpts from the Constitution of Kenya to answer the question:
{context}

Question: {question}
Answer:"""

    data = {
        "model": "mixtral-8x7b-32768",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"]

In [18]:
def ask_constitution(query, api_key):
    context = retrieve_context(query)
    answer = query_groq(context, query, api_key)
    return answer



In [21]:
# Example Usage 
import requests
def query_groq(context, query, api_key):
    url = "https://api.groq.com/openai/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    prompt = f"""
    Based on the following context from the Constitution, answer the question.
    
    Context: {context}
    
    Question: {query}
    
    Answer:
    """
    
    data = {
        "model": "llama-3.3-70b-versatile",  # Updated model name
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code != 200:
        return f"Error: API request failed with status {response.status_code}"
    
    response_data = response.json()
    
    if "choices" not in response_data:
        return f"Error: Unexpected API response format. Response: {response_data}"
    
    return response_data["choices"][0]["message"]["content"]

In [29]:
groq_api_key = "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"
question = input("Enter your question about the Constitution of Kenya: ")
response = ask_constitution(question, groq_api_key)
print(response)

According to the Constitution of Kenya, 2010, the following points are made about tax:

1. **National Government Taxes**: The national government has the power to impose the following taxes:
   - Income tax
   - Value-added tax
   - Customs duties and other duties on import and export goods
   - Excise tax

2. **Authorization for Other Taxes**: An Act of Parliament may authorize the national government to impose any other tax or duty, except for property rates and entertainment taxes, which are reserved for counties.

3. **County Government Taxes**: A county may impose:
   - Property rates
   - Entertainment taxes
   - Any other tax that it is authorized to impose by an Act of Parliament

4. **Restrictions on County Taxation Powers**: The taxation and other revenue-raising powers of a county shall not be exercised in a way that prejudices national economic policies, economic activities across county boundaries, or the national mobility of goods, services, capital, or labor.

5. **Charg